# Star Type Classification / NASA

NASA에서 수집한 별의 유형 분류 DataSet이다.

## Star Type Classification


For comparing all models of ML   
It can be used for prediction

Temperature -- K   
L -- L/Lo  
R -- R/Ro  
AM -- Mv  
Color -- General Color of Spectrum  
Spectral_Class -- O,B,A,F,G,K,M / SMASS - https://en.wikipedia.org/wiki/Asteroid_spectral_types  
Type -- Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , Super Giants, Hyper Giants  

TARGET:  
Type

from 0 to 5  

Red Dwarf - 0  
Brown Dwarf - 1  
White Dwarf - 2  
Main Sequence - 3  
Super Giants - 4  
Hyper Giants - 5

MATH:  

Lo = 3.828 x 10^26 Watts  
(Avg Luminosity of Sun)  
Ro = 6.9551 x 10^8 m  
(Avg Radius of Sun)  

## 요약
 요약
Calumns: 온도(K), 광도(L), 반지름(R), AM, 색상, 스펙트럼(Spectral Class), Type (분류)  

![Spectral_Class](https://scienceruls.weebly.com/uploads/5/1/7/4/51741831/846626525.GIF?448)

참고는 Kaggle에서 (https://www.kaggle.com/sohelranaccselab/star-type-classification-using-machine-learning)

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

In [3]:
DATA_PATH = os.path.join('data') # 데이터 저장 폴더

def load_star_data(): # wine data의 csv파일을 return하는 함수
    csv_path = os.path.join(DATA_PATH,'Stars.csv')
    return pd.read_csv(csv_path)

In [4]:
star = load_star_data()
star

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5
237,8829,537493.000000,1423.0000,-10.73,White,A,5
238,9235,404940.000000,1112.0000,-11.23,White,A,5


In [5]:
# Stars DataSet은 String 형식의 Data가 존재해서 데이터 스케일링이 불가능하다. 즉 convert numeric to categorical을 해줘야 한다.
# Data를 Numeric하게 바꾸는 작업이 필요한 column은 Color와 Spectral_Class이다.
star.info()
star.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature     240 non-null    int64  
 1   L               240 non-null    float64
 2   R               240 non-null    float64
 3   A_M             240 non-null    float64
 4   Color           240 non-null    object 
 5   Spectral_Class  240 non-null    object 
 6   Type            240 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 13.2+ KB


,count,mean,std,min,25%,50%,75%,max
Temperature,240.0,10497.462500,9552.425037,1939.00000,3344.250000,5776.0000,15055.5000,40000.00
L,240.0,107188.361635,179432.244940,0.00008,0.000865,0.0705,198050.0000,849420.00
R,240.0,237.157781,517.155763,0.00840,0.102750,0.7625,42.7500,1948.50
A_M,240.0,4.382396,10.532512,-11.92000,-6.232500,8.3130,13.6975,20.06
Type,240.0,2.500000,1.711394,0.00000,1.000000,2.5000,4.0000,5.00


In [6]:
# 데이터를 보면 같은 데이터인데 구분을 이상하게 한 모습을 볼수 있다. 이것을 7개 정도로 추려보자.
star.Color.value_counts()

Red                   112
Blue                   56
Blue-white             26
Blue White             10
yellow-white            8
White                   7
Blue white              4
Yellowish White         3
white                   3
yellowish               2
Orange                  2
Whitish                 2
Blue-White              1
Pale yellow orange      1
White-Yellow            1
Yellowish               1
Orange-Red              1
Name: Color, dtype: int64

In [7]:
color_dict = {
    'Red' : 'Red', 
    'Blue' : 'Blue', 
    'Blue-white' : 'Blue-white', 
    'Blue White' : 'Blue-white', 
    'yellow-white' : 'White-Yellow', 
    'White' : 'White',
    'Blue white' : 'Blue-white', 
    'white' : 'White', 
    'Yellowish White' : 'White-Yellow', 
    'yellowish' : 'Yellow', 
    'Orange' : 'Orange',
    'Whitish' : 'White', 
    'Yellowish' : 'Yellow', 
    'Blue-White' : 'Blue-white', 
    'Pale yellow orange' : 'Orange',
    'Orange-Red' : 'Orange', 
    'White-Yellow' : 'White-Yellow' 
}

star.Color = star.Color.map(color_dict).astype('category')
star.Color.value_counts()
star.groupby('Color').describe()
# 이제 Color의 종류를 7가지로 분류 했다.

Temperature                                               \
                   count          mean          std     min       25%   
Color                                                                   
Blue                56.0  21918.339286  9453.979609  5752.0  14797.75   
Blue-white          41.0  16659.951220  6560.050964  8570.0  11900.00   
Orange               4.0   5094.500000  1530.042810  3749.0   4152.50   
Red                112.0   3291.785714   490.780761  1939.0   2987.50   
White               12.0   9579.583333  2368.265513  7220.0   7974.00   
White-Yellow        12.0   7609.166667  2428.134444  5300.0   5902.00   
Yellow               3.0   4527.666667   451.502307  4077.0   4301.50   

                                              L                 ...      A_M  \
                  50%       75%      max  count           mean  ...      75%   
Color                                                           ...            
Blue          19921.5  27489.50  40000.0   56.0  249974.732446  ...  -4.3900   
Blue-white    14100.0  22012.00  30000.0   41.0   68904.416521  ...  11.3800   
Orange         4699.5   5641.50   7230.0    4.0  295000.157520  ...   7.0300   
Red            3324.0   3550.75   6850.0  112.0   60526.787893  ...  16.8900   
White          8879.5   9918.50  14732.0   12.0   78542.975191  ...  14.0725   
White-Yellow   6928.5   7852.50  12990.0   12.0       1.960553  ...  12.3200   
Yellow         4526.0   4753.00   4980.0    3.0       0.198333  ...   6.3670   

                       Type                                                
                 max  count      mean       std  min   25%  50%  75%  max  
Color                                                                      
Blue          12.900   56.0  3.571429  0.988243  2.0  3.00  4.0  4.0  5.0  
Blue-white    14.870   41.0  2.951220  0.973427  2.0  2.00  3.0  3.0  5.0  
Orange        14.080    4.0  3.750000  1.500000  2.0  2.75  4.0  5.0  5.0  
Red           20.060  112.0  1.705357  1.975701  0.0  0.00  1.0  4.0  5.0  
White         14.810   12.0  2.666667  1.154701  2.0  2.00  2.0  3.0  5.0  
White-Yellow  14.470   12.0  2.666667  0.492366  2.0  2.00  3.0  3.0  3.0  
Yellow         6.506    3.0  3.000000  0.000000  3.0  3.00  3.0  3.0  3.0  

[7 rows x 40 columns]

In [8]:
star.Spectral_Class.value_counts()

M    111
B     46
O     40
A     19
F     17
K      6
G      1
Name: Spectral_Class, dtype: int64

In [9]:
# 사이킷 런의 preprocessing중 LabelEncoder이라는 것이 있다. 이를 적용해서 Object 형식의 값들을 Int 형식으로 형변환 해보겠다.

le = preprocessing.LabelEncoder()

selected_columns = ['Color','Spectral_Class']

le.fit(star[selected_columns].values.flatten())
star[selected_columns] = star[selected_columns].apply(le.fit_transform)
star.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature     240 non-null    int64  
 1   L               240 non-null    float64
 2   R               240 non-null    float64
 3   A_M             240 non-null    float64
 4   Color           240 non-null    int32  
 5   Spectral_Class  240 non-null    int32  
 6   Type            240 non-null    int64  
dtypes: float64(3), int32(2), int64(2)
memory usage: 11.4 KB


In [10]:
star
# 이렇게 다듬어진 DataSet을 Normalize 시켜보자.

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,3,5,0
1,3042,0.000500,0.1542,16.60,3,5,0
2,2600,0.000300,0.1020,18.70,3,5,0
3,2800,0.000200,0.1600,16.65,3,5,0
4,1939,0.000138,0.1030,20.06,3,5,0
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,0,6,5
236,30839,834042.000000,1194.0000,-10.63,0,6,5
237,8829,537493.000000,1423.0000,-10.73,4,0,5
238,9235,404940.000000,1112.0000,-11.23,4,0,5


In [19]:
norm = Normalizer()
trans_star = norm.fit_transform(star)
pd.DataFrame(trans_star,columns=star.columns)

#Normalizer를 써봤는데 Color와 같이 값이 같은 것들에 대해서 같은 값을 보이지 않는다. 다른 스케일링 기법을 사용해 봐야겠다.

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,0.999984,7.822564e-07,0.000055,0.005254,0.000978,0.001630,0.000000
1,0.999983,1.643628e-07,0.000051,0.005457,0.000986,0.001644,0.000000
2,0.999972,1.153813e-07,0.000039,0.007192,0.001154,0.001923,0.000000
3,0.999980,7.142715e-08,0.000057,0.005946,0.001071,0.001786,0.000000
4,0.999942,7.116658e-08,0.000053,0.010345,0.001547,0.002578,0.000000
...,...,...,...,...,...,...,...
235,0.103330,9.946406e-01,0.003598,-0.000026,0.000000,0.000016,0.000013
236,0.036950,9.993161e-01,0.001431,-0.000013,0.000000,0.000007,0.000006
237,0.016424,9.998616e-01,0.002647,-0.000020,0.000007,0.000000,0.000009
238,0.022800,9.997363e-01,0.002745,-0.000028,0.000010,0.000000,0.000012


In [22]:
stand=StandardScaler()
trans_star = stand.fit_transform(star)
pd.DataFrame(trans_star,columns=star.columns)

# StandardScaler를 사용하니 같은 값을 가진 값들이 같은 값을 가지게 되었다.

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,-0.779382,-0.598624,-0.459210,1.116745,0.561635,0.595338,-1.46385
1,-0.782110,-0.598624,-0.459241,1.162414,0.561635,0.595338,-1.46385
2,-0.828477,-0.598624,-0.459342,1.362213,0.561635,0.595338,-1.46385
3,-0.807496,-0.598624,-0.459229,1.167171,0.561635,0.595338,-1.46385
4,-0.897819,-0.598624,-0.459340,1.491607,0.561635,0.595338,-1.46385
...,...,...,...,...,...,...,...
235,2.983743,1.494720,2.167974,-1.361718,-1.373185,1.074805,1.46385
236,2.133913,4.059319,1.854068,-1.428317,-1.373185,1.074805,1.46385
237,-0.175029,2.403157,2.297800,-1.437832,1.206575,-1.801997,1.46385
238,-0.132438,1.662878,1.695177,-1.485403,1.206575,-1.801997,1.46385
